# Hand Count: Real-Time Finger Detection Using Computer Vision

In [1]:
import cv2
import mediapipe as mp

C:\Users\User\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

In [3]:
def count_fingers(hand_landmarks, hand_label):
    finger_tips = [8, 12, 16, 20]
    fingers_up = []

    for tip_id in finger_tips:
        tip = hand_landmarks.landmark[tip_id]
        dip = hand_landmarks.landmark[tip_id - 2]

        if tip.y < dip.y:
            fingers_up.append(1)
        else:
            fingers_up.append(0)

    thumb_tip = hand_landmarks.landmark[4]
    thumb_knuckle = hand_landmarks.landmark[2]

    if hand_label == 'Right':
        if thumb_tip.x < thumb_knuckle.x:
            fingers_up.append(1)
        else:
            fingers_up.append(0)
    else:
        if thumb_tip.x > thumb_knuckle.x:
            fingers_up.append(1)
        else:
            fingers_up.append(0)

    return fingers_up.count(1)

previous_count = None
previous_previous_count = None

In [4]:
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame_rgb)

    total_fingers = 0

    if result.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            hand_label = handedness.classification[0].label
            total_fingers += count_fingers(hand_landmarks, hand_label)

    if not result.multi_hand_landmarks:
        if previous_count is not None and previous_previous_count is not None:
            output = previous_count + previous_previous_count
            cv2.putText(frame, f'Total Fingers: {output} (previous: {previous_count})', 
                        (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, f'{previous_previous_count} + {previous_count} = {output}', 
                        (frame.shape[1] - 300, frame.shape[0] - 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, 'Total Fingers: 0', (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    else:
        if previous_count is None or previous_count != total_fingers:
            previous_previous_count = previous_count
            previous_count = total_fingers

        cv2.putText(frame, f'Total Fingers: {total_fingers}', (10, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("Hand Number Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\User\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
